## Este es un cuaderno generador de archivos .csv con reportes por formato que considera la variable de tipo de resurtible. 
El dataset de 8 semanas no tiene el tipo de resurtible. 

In [10]:
# Importar librerías básicas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import researchpy as rp

In [11]:
from sklearn import preprocessing
from sklearn import model_selection
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import neighbors
from catboost import CatBoostClassifier, Pool, metrics, cv
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

In [12]:
# Especificar columnas
relevant_columns = ['SKU','Tienda','FORMATO','REGION','Categoria','Proveedor','CLASE','Mes','DiaSem','Sem','TipoResurtible','Existencia','fiDesplazamiento','VentaPromedio','Y_Faltante']
num_columns = ['Existencia','fiDesplazamiento','VentaPromedio']
cat_columns = ['SKU','Tienda','FORMATO','REGION','Categoria','Proveedor','CLASE','Mes','DiaSem','Sem']
target = ['Y_Faltante']

In [13]:
# Abrir el archivo que contiene el conjunto de validación 
file_val = 'datos_prueba.csv'
data_val = pd.read_csv(file_val, header=0,low_memory=False, encoding='latin-1')
print('rows:', data_val.shape[0], ' columns:', data_val.shape[1])

rows: 842788  columns: 25


In [14]:
data_val_limpio = data_val.dropna(subset=relevant_columns)

In [15]:
data_val_limpio.to_csv('validacion_limpio.csv',index=True)

In [16]:
dtype = {'SKU':'object','FORMATO':'object','REGION':'object','Tienda':'object','Categoria':'object','Proveedor':'object','CLASE':'object','SCLASE':'object','Mes':'object','DiaSem':'object','Sem':'object','Decil':'object','TipoResurtible':'object','Existencia':'float','fiDesplazamiento':'float',
\
'VentaPromedio':'float','Y_Faltante':'int'}

In [17]:
# Abrir el archivo
file = 'datos_entrenamiento.csv'
data = pd.read_csv(file, header=0,low_memory=False, encoding='latin-1',usecols=relevant_columns,dtype=dtype)
print('rows:', data.shape[0], ' columns:', data.shape[1])

rows: 5094322  columns: 15


In [18]:
# Abrir el archivo
file = 'datos_entrenamiento_2.csv'
data_2 = pd.read_csv(file, header=0,low_memory=False, encoding='latin-1',usecols=relevant_columns,dtype=dtype)
print('rows:', data_2.shape[0], ' columns:', data_2.shape[1])

ValueError: Usecols do not match columns, columns expected but not found: ['TipoResurtible']

In [ ]:
# Abrir el archivo que contiene el conjunto de validación 
file_val = 'validacion_limpio.csv'
#file_val = 'datos_prueba.csv'
data_val = pd.read_csv(file_val, header=0,low_memory=False, encoding='latin-1',usecols=relevant_columns,dtype=dtype)
print('rows:', data_val.shape[0], ' columns:', data_val.shape[1])

rows: 808817  columns: 14


In [ ]:
data = pd.concat([data,data_2],join='inner')

In [ ]:
data_val = data_val.sample(frac=1).reset_index(drop=True)
data_val

,SKU,Tienda,FORMATO,REGION,Categoria,Proveedor,CLASE,Mes,DiaSem,Sem,Existencia,fiDesplazamiento,VentaPromedio,Y_Faltante
0,3018195,CHEDRAUI SELECTO XALAPA ANIMAS,CHEDRAUI SELECTO,XALAPA,PAN,BIMBO SA DE CV,PAN,6,3,26,1560.0,4863.0,167773.50,0
1,3041748,CHEDRAUI MEXICO ARAGON NORTE,CHEDRAUI CD,VM ORIENTE,JUGOS Y NÃCTARES,PROPIMEX S DE RL DE CV,JUGOS Y NÃCTARES,6,7,26,104.0,289.0,3294.60,0
2,3108795,CHEDRAUI MEXICO NEZA ESTADIO,CHEDRAUI CD,VM CENTRO,ARTÃCULOS DE COCCIÃN,CINSA SA DE CV,ARTÃCULOS DE COCCIÃN,6,2,27,6.0,11.0,2740.87,1
3,3090425,CHEDRAUI POZA RICA CRYSTAL,CHEDRAUI AB,XALAPA,TORTILLAS EMPACADAS,MISSION FOODS MEXICO S DE RL D (35059),TORTILLAS EMPACADAS,6,2,26,37.0,112.0,3024.00,0
4,3679979,CHEDRAUI VILLA TAMULTE,CHEDRAUI CD,VILLAHERMOSA,DULCES Y CHOCOLATES,ALEJANDRO CAMARGO MONROY (41743),DULCES Y CHOCOLATES,6,1,26,17.0,25.0,438.25,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
808812,3332921,SUPER CHEDRAUI XAL. 20 DE NOVIEMBRE,SUPER CHEDRAUI CD,XALAPA,BOTANAS,COMERCIALIZADORA PEPSICO MEXICO S,BOTANAS,7,4,29,21.0,49.0,1430.31,0
808813,3121110,CHEDRAUI OAXACA MADERO,CHEDRAUI CD,CORDOBA,CERVEZAS,CERVECERIA MODELO DE MEXICO S (35594),CERVEZAS,7,2,28,36.0,81.0,10055.34,0
808814,3064080,CHEDRAUI OAXACA XOXOCOTLAN,CHEDRAUI CD,CORDOBA,FOCOS,INDUSTRIAS ADIR SA DE CV (35519),FOCOS,7,6,28,10.0,0.0,0.00,1
808815,3720043,CHEDRAUI COMALCALCO NORTE,CHEDRAUI CD,TABASCO,FÃRMULAS INFANTILES,MARCAS NESTLE SA DE CV,FÃRMULAS INFANTILES,7,3,28,18.0,20.0,7520.00,0


In [ ]:
# Aislamos las variables numéricas y realizamos un min-max scaling sobre ellas
categorical = data.drop(columns=['Existencia','fiDesplazamiento','VentaPromedio','Y_Faltante'])
numerical = data[['Existencia','fiDesplazamiento','VentaPromedio']]
numerical_scaled = (numerical - numerical.min())/(numerical.max() - numerical.min()) 
faltante = data["Y_Faltante"]

In [ ]:
scaled_data = pd.concat([categorical,numerical_scaled,faltante], axis=1)
data = scaled_data

In [ ]:
data = data.dropna()
data_val = data_val.dropna()

In [ ]:
data.dtypes

SKU                  object
Tienda               object
FORMATO              object
REGION               object
Categoria            object
Proveedor            object
CLASE                object
Mes                  object
DiaSem               object
Sem                  object
Existencia          float64
fiDesplazamiento    float64
VentaPromedio       float64
Y_Faltante            int64
dtype: object

In [ ]:
def segmentador_formato(frame,cadena):
    arreglo = frame[frame['FORMATO']==cadena]
    return arreglo 

In [ ]:
X_train = data.drop('Y_Faltante',axis=1)
y_train = data.Y_Faltante
X_val = data_val.drop('Y_Faltante',axis=1)
y_val = data_val.Y_Faltante

In [ ]:
indices_categoricos = np.where(X_train.dtypes != float)[0]
indices_categoricos

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
num_ind = np.where(X_train.dtypes != object)[0]
num_ind

array([10, 11, 12])

In [ ]:
model = CatBoostClassifier(
    l2_leaf_reg=int(2.0),
    custom_loss=[metrics.Accuracy()],
    random_seed=0,
    logging_level='Verbose', # Write 'Silent' instead of 'verbose'
    task_type='GPU',
    iterations=240
)

In [ ]:
model.fit(
    X_train, y_train,
    cat_features=indices_categoricos,
    eval_set=(X_val, y_val),
    logging_level='Verbose',  # you can uncomment this for text output
    plot=True
);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6497623	test: 0.6487143	best: 0.6487143 (0)	total: 1.07s	remaining: 17m 50s
1:	learn: 0.6111399	test: 0.6099043	best: 0.6099043 (1)	total: 2.06s	remaining: 17m 10s
2:	learn: 0.5752609	test: 0.5741597	best: 0.5741597 (2)	total: 3.01s	remaining: 16m 39s
3:	learn: 0.5427705	test: 0.5413786	best: 0.5413786 (3)	total: 3.98s	remaining: 16m 30s
4:	learn: 0.5140664	test: 0.5127139	best: 0.5127139 (4)	total: 4.96s	remaining: 16m 27s
5:	learn: 0.4884752	test: 0.4870582	best: 0.4870582 (5)	total: 6.06s	remaining: 16m 43s
6:	learn: 0.4653077	test: 0.4643028	best: 0.4643028 (6)	total: 7.27s	remaining: 17m 10s
7:	learn: 0.4447389	test: 0.4435435	best: 0.4435435 (7)	total: 8.44s	remaining: 17m 26s
8:	learn: 0.4263066	test: 0.4241650	best: 0.4241650 (8)	total: 9.52s	remaining: 17m 28s
9:	learn: 0.4092822	test: 0.4072885	best: 0.4072885 (9)	total: 10.8s	remaining: 17m 45s
10:	learn: 0.3960744	test: 0.3943219	best: 0.3943219 (10)	total: 11.7s	remaining: 17m 35s
11:	learn: 0.3820753	test: 0.3

In [ ]:
y_pred = model.predict(X_val)
prediction_probs = model.predict_proba(X_val)

accuracy = accuracy_score(y_val,y_pred)
recall = recall_score(y_val,y_pred,average='binary',pos_label=1)
precision = precision_score(y_val,y_pred,average='binary',pos_label=0)
balanced_accuracy = balanced_accuracy_score(y_val,y_pred)
f1 = f1_score(y_val,y_pred,pos_label=1)
print(f'Accuracy: {accuracy}')
print(f'True Positive Rate {recall}')
print(f'Precision {precision}')
print(f'Balanced accuracy {balanced_accuracy}')
print(f'f1 score {f1}')

Accuracy: 0.9200968822366493
True Positive Rate 0.8810620133591558
Precision 0.889485160786834
Balanced accuracy 0.920136255181438
f1 score 0.9169279258782231


## Exportamos nuestros datos a .csv

In [ ]:
data_val_limpio['Ago_Prediccion'] = model.predict(X_val)

In [ ]:
data_val_limpio

,fiIdTienda,fiEvento,fiPeriodo,SKU,Tienda,FORMATO,REGION,Categoria,Proveedor,DEPTO,...,Decil,Importado,MarcaPropia,EnOferta,TipoResurtible,Existencia,fiDesplazamiento,VentaPromedio,Y_Faltante,Ago_Prediccion
0,233,128,202206,3269530,CHEDRAUI MEXICO COAPA,CHEDRAUI AB,VM SUR,JUGOS Y NÉCTARES,BEBIDAS PURIFICADAS S DE RL DE CV,ABARROTES,...,9,1,0,0,RL,43,29,290.00,0,0
2,233,128,202206,3003153,CHEDRAUI MEXICO COAPA,CHEDRAUI AB,VM SUR,PAÑALES,PRODUCTOS INTERNACIONALES MABESA SA,ABARROTES,...,9,1,0,0,RC,0,13,1534.00,1,0
3,233,128,202206,3406681,CHEDRAUI MEXICO COAPA,CHEDRAUI AB,VM SUR,FOCOS,AKSI HERRAMIENTAS SA DE CV,NaN,...,9,1,0,0,RC,13,4,176.00,1,1
4,233,128,202206,3657942,CHEDRAUI MEXICO COAPA,CHEDRAUI AB,VM SUR,JABONES DE TOCADOR,COLGATE PALMOLIVE SA DE CV,ABARROTES,...,9,1,0,1,RC,0,32,704.00,0,0
5,233,128,202206,3269493,CHEDRAUI MEXICO COAPA,CHEDRAUI AB,VM SUR,FOCOS,ENERGIZER M+XICO S DE RL DE CV (35132),NaN,...,9,1,0,0,RC,27,7,1146.46,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
842783,678,39,202207,3005047,SUPERCITO EJE 4 NORTE,SUPERCITO CD,VM SELECTO,TOALLAS FEMENINAS,KIMBERLY CLARK DE MEXICO SA DE CV,ABARROTES,...,9,1,0,0,RC,26,9,205.65,0,0
842784,678,39,202207,3004779,SUPERCITO EJE 4 NORTE,SUPERCITO CD,VM SELECTO,TOALLAS FEMENINAS,SCA CONSUMIDOR MEXICO SA DE CV,ABARROTES,...,5,1,0,0,RC,0,22,0.00,1,0
842785,678,39,202207,3298737,SUPERCITO EJE 4 NORTE,SUPERCITO CD,VM SELECTO,LIMPIEZA CALZADO,KW EMPAQUES SA DE CV (35817),ABARROTES,...,9,1,0,0,RC,0,0,0.00,1,1
842786,678,39,202207,3724930,SUPERCITO EJE 4 NORTE,SUPERCITO CD,VM SELECTO,GALLETAS,BIMBO SA DE CV,ABARROTES,...,4,1,0,1,RL,0,32,0.00,1,0


In [ ]:
data_val_limpio.to_csv('Resurtibles_Resumen_modelo_entrenado_general.csv',index=True)

## Reporte por Formato 

In [ ]:
def reporte_formato(frame,frame_val,data_val_limpio):
    l = len(frame['FORMATO'].value_counts())
    formatos = frame['FORMATO'].value_counts()
    def segmentador_formato(frame,cadena):
        arreglo = frame[frame['FORMATO']==cadena]
        return arreglo
    lineas = []
    dict_acc = {}
    dict_balanced = {}
    dict_recall = {}
    dict_pre = {}
    dict_f1 = {}
    dict_graph = {}
    dict_frame = {}
    for i in range(l):
        cadena = formatos.iloc[i:i+1].index[0]
        formato_train = segmentador_formato(frame,cadena)
        formato_val = segmentador_formato(frame_val,cadena)
        syntetic = 2*(len(formato_train)*len(formato_val))/(len(formato_train)+len(formato_val))
        X_train = formato_train.drop('Y_Faltante',axis=1)
        y_train = formato_train.Y_Faltante
        X_val = formato_val.drop('Y_Faltante',axis=1)
        y_val = formato_val.Y_Faltante
        cat = CatBoostClassifier(
            l2_leaf_reg=int(2.0),
            custom_loss=[metrics.Accuracy()],
            random_seed=0,
            logging_level='Verbose',
            task_type='GPU',
            #iterations=50
        )
        cat.fit(
            X_train,y_train,
            cat_features=indices_categoricos,
            eval_set=(X_val,y_val),
            logging_level='Verbose',
            plot=True
        );
        y_pred = cat.predict(X_val)
        y_prob = cat.predict_proba(X_val)
        pred_cb = pd.DataFrame(y_prob).assign(CB_Pred=pd.Series(y_pred))
        pred_cb = pd.DataFrame(pred_cb).assign(TrueValue=pd.Series(y_val))
        accuracy = accuracy_score(y_val,y_pred)
        recall = recall_score(y_val,y_pred)
        precision = precision_score(y_val,y_pred)
        balanced_accuracy = balanced_accuracy_score(y_val,y_pred)
        f1 = f1_score(y_val,y_pred)
        lineas.append(f'###############')
        lineas.append(f'')
        lineas.append(f'Nombre de formato :  ' + cadena)
        lineas.append(f'Precisión: {accuracy} ')
        lineas.append(f'Precisión Balanceada: {balanced_accuracy} ')
        lineas.append(f'Recall: {recall} ')
        lineas.append(f'Precision: {precision} ')
        lineas.append(f'Calificación F1: {f1} ')
        lineas.append(f'')
        dict_acc[cadena]=accuracy
        dict_recall[cadena]=recall 
        dict_pre[cadena]=precision
        dict_balanced[cadena]=balanced_accuracy
        dict_graph[cadena]=(syntetic,balanced_accuracy)
        dict_frame[cadena]=pred_cb
        dict_f1[cadena]=f1
        data_val_limpio_formato = segmentador_formato(data_val_limpio,cadena)
        formato_target = segmentador_formato(X_val,cadena)
        data_val_limpio_formato['Ago_Prediccion'] = cat.predict(formato_target)
        cadena_2 = f'resurtible_' + cadena + f'.csv'
        data_val_limpio_formato.to_csv(cadena_2,index=True)
    return [lineas,dict_balanced,dict_graph,dict_frame,dict_acc,dict_recall,dict_pre,dict_f1]

In [ ]:
resultados_formato=reporte_formato(data,data_val,data_val_limpio)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6458986	test: 0.6430477	best: 0.6430477 (0)	total: 351ms	remaining: 5m 50s
1:	learn: 0.6039025	test: 0.6083191	best: 0.6083191 (1)	total: 754ms	remaining: 6m 16s
2:	learn: 0.5644944	test: 0.5682655	best: 0.5682655 (2)	total: 1.12s	remaining: 6m 11s
3:	learn: 0.5307544	test: 0.5337289	best: 0.5337289 (3)	total: 1.6s	remaining: 6m 39s
4:	learn: 0.5001057	test: 0.5025391	best: 0.5025391 (4)	total: 2.04s	remaining: 6m 45s
5:	learn: 0.4726886	test: 0.4744980	best: 0.4744980 (5)	total: 2.5s	remaining: 6m 53s
6:	learn: 0.4485175	test: 0.4498218	best: 0.4498218 (6)	total: 2.87s	remaining: 6m 46s
7:	learn: 0.4291219	test: 0.4310873	best: 0.4310873 (7)	total: 3.32s	remaining: 6m 51s
8:	learn: 0.4097869	test: 0.4113817	best: 0.4113817 (8)	total: 3.69s	remaining: 6m 46s
9:	learn: 0.3928716	test: 0.3931774	best: 0.3931774 (9)	total: 4.07s	remaining: 6m 43s
10:	learn: 0.3773773	test: 0.3771038	best: 0.3771038 (10)	total: 4.44s	remaining: 6m 39s
11:	learn: 0.3639347	test: 0.3632907	best: 

/tmp/ipykernel_83541/2097710240.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_val_limpio_formato['Ago_Prediccion'] = cat.predict(formato_target)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6492485	test: 0.6486535	best: 0.6486535 (0)	total: 203ms	remaining: 3m 22s
1:	learn: 0.6093934	test: 0.6091584	best: 0.6091584 (1)	total: 444ms	remaining: 3m 41s
2:	learn: 0.5729074	test: 0.5724575	best: 0.5724575 (2)	total: 624ms	remaining: 3m 27s
3:	learn: 0.5400510	test: 0.5402326	best: 0.5402326 (3)	total: 817ms	remaining: 3m 23s
4:	learn: 0.5100181	test: 0.5115452	best: 0.5115452 (4)	total: 1.01s	remaining: 3m 20s
5:	learn: 0.4833161	test: 0.4854820	best: 0.4854820 (5)	total: 1.19s	remaining: 3m 17s
6:	learn: 0.4608495	test: 0.4669183	best: 0.4669183 (6)	total: 1.38s	remaining: 3m 15s
7:	learn: 0.4419291	test: 0.4514279	best: 0.4514279 (7)	total: 1.57s	remaining: 3m 14s
8:	learn: 0.4226548	test: 0.4327534	best: 0.4327534 (8)	total: 1.75s	remaining: 3m 12s
9:	learn: 0.4059227	test: 0.4162662	best: 0.4162662 (9)	total: 1.97s	remaining: 3m 14s
10:	learn: 0.3904730	test: 0.4007914	best: 0.4007914 (10)	total: 2.15s	remaining: 3m 13s
11:	learn: 0.3765176	test: 0.3867078	best

...

0:	learn: 0.6574843	test: 0.6563796	best: 0.6563796 (0)	total: 20.5ms	remaining: 20.5s
1:	learn: 0.6257988	test: 0.6245263	best: 0.6245263 (1)	total: 39.3ms	remaining: 19.6s
2:	learn: 0.5978300	test: 0.5960014	best: 0.5960014 (2)	total: 57.2ms	remaining: 19s
3:	learn: 0.5737633	test: 0.5707695	best: 0.5707695 (3)	total: 77.4ms	remaining: 19.3s
4:	learn: 0.5506944	test: 0.5463420	best: 0.5463420 (4)	total: 94.4ms	remaining: 18.8s
5:	learn: 0.5311187	test: 0.5254223	best: 0.5254223 (5)	total: 114ms	remaining: 18.9s
6:	learn: 0.5100966	test: 0.5052438	best: 0.5052438 (6)	total: 132ms	remaining: 18.7s
7:	learn: 0.4916744	test: 0.4875764	best: 0.4875764 (7)	total: 149ms	remaining: 18.5s
8:	learn: 0.4724331	test: 0.4713093	best: 0.4713093 (8)	total: 168ms	remaining: 18.5s
9:	learn: 0.4579431	test: 0.4568584	best: 0.4568584 (9)	total: 186ms	remaining: 18.4s
10:	learn: 0.4415344	test: 0.4446614	best: 0.4446614 (10)	total: 205ms	remaining: 18.5s
11:	learn: 0.4282857	test: 0.4317849	best: 0.4317

/tmp/ipykernel_83541/2097710240.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_val_limpio_formato['Ago_Prediccion'] = cat.predict(formato_target)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6417012	test: 0.6533747	best: 0.6533747 (0)	total: 15.1ms	remaining: 15.1s
1:	learn: 0.5951268	test: 0.6174849	best: 0.6174849 (1)	total: 31.4ms	remaining: 15.7s
2:	learn: 0.5525306	test: 0.5841448	best: 0.5841448 (2)	total: 46.7ms	remaining: 15.5s
3:	learn: 0.5164814	test: 0.5553972	best: 0.5553972 (3)	total: 62ms	remaining: 15.4s
4:	learn: 0.4833537	test: 0.5284534	best: 0.5284534 (4)	total: 77.6ms	remaining: 15.4s
5:	learn: 0.4542260	test: 0.5037938	best: 0.5037938 (5)	total: 92.1ms	remaining: 15.3s
6:	learn: 0.4282800	test: 0.4814207	best: 0.4814207 (6)	total: 106ms	remaining: 15.1s
7:	learn: 0.4060274	test: 0.4606889	best: 0.4606889 (7)	total: 122ms	remaining: 15.1s
8:	learn: 0.3841114	test: 0.4411890	best: 0.4411890 (8)	total: 140ms	remaining: 15.4s
9:	learn: 0.3659500	test: 0.4235131	best: 0.4235131 (9)	total: 155ms	remaining: 15.4s
10:	learn: 0.3478151	test: 0.4093619	best: 0.4093619 (10)	total: 174ms	remaining: 15.6s
11:	learn: 0.3322175	test: 0.3960638	best: 0.396

/tmp/ipykernel_83541/2097710240.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_val_limpio_formato['Ago_Prediccion'] = cat.predict(formato_target)


In [ ]:
resultados_formato[0]

['###############',
 '',
 'Nombre de formato :  CHEDRAUI CD',
 'Precisión: 0.9307335831197002 ',
 'Precisión Balanceada: 0.9308071253429282 ',
 'Recall: 0.9031255292530089 ',
 'Precision: 0.9562785262658575 ',
 'Calificación F1: 0.9289423109342864 ',
 '',
 '###############',
 '',
 'Nombre de formato :  CHEDRAUI AB',
 'Precisión: 0.9124035614963667 ',
 'Precisión Balanceada: 0.9123819767154212 ',
 'Recall: 0.8696076561465708 ',
 'Precision: 0.9509139982824193 ',
 'Calificación F1: 0.9084452153937214 ',
 '',
 '###############',
 '',
 'Nombre de formato :  SUPER CHEDRAUI CD',
 'Precisión: 0.9087184156612793 ',
 'Precisión Balanceada: 0.9087537226332975 ',
 'Recall: 0.8503381081632333 ',
 'Precision: 0.9628695281409893 ',
 'Calificación F1: 0.9031118516975631 ',
 '',
 '###############',
 '',
 'Nombre de formato :  CHEDRAUI SELECTO',
 'Precisión: 0.9300176379804512 ',
 'Precisión Balanceada: 0.9300153464252983 ',
 'Recall: 0.9092278857814854 ',
 'Precision: 0.9486579754601226 ',
 'Calificac

: 